In [ ]:
! wget http://www.statmt.org/europarl/v7/fr-en.tgz

--2023-02-28 07:48:30--  http://www.statmt.org/europarl/v7/fr-en.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.197.184
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.statmt.org/europarl/v7/fr-en.tgz [following]
--2023-02-28 07:48:30--  https://www.statmt.org/europarl/v7/fr-en.tgz
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202718517 (193M) [application/x-gzip]
Saving to: ‘fr-en.tgz’

fr-en.tgz           100%[===================>] 193.33M  2.22MB/s    in 88s     

2023-02-28 07:49:59 (2.20 MB/s) - ‘fr-en.tgz’ saved [202718517/202718517]



In [ ]:
! tar zxvf fr-en.tgz

europarl-v7.fr-en.en
europarl-v7.fr-en.fr


In [ ]:
! head europarl-v7.fr-en.en

Resumption of the session
I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
You have requested a debate on this subject in the course of the next few days, during this part-session.
In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.
Please rise, then, for this minute' s silence.
(The House rose and observed a minute' s silence)
Madam President, on a point of order.
You will be aware from the press and television that there have been a number of bomb explosions and

In [ ]:
! head europarl-v7.fr-en.fr

Reprise de la session
Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.
Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit. En revanche, les citoyens d'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.
Vous avez souhaité un débat à ce sujet dans les prochains jours, au cours de cette période de session.
En attendant, je souhaiterais, comme un certain nombre de collègues me l'ont demandé, que nous observions une minute de silence pour toutes les victimes, des tempêtes notamment, dans les différents pays de l'Union européenne qui ont été touchés.
Je vous invite à vous lever pour cette minute de silence.
(Le Parlement, debout, observe une minute de silence)
Madame la Présidente, c'est une motion de procédure.
Vous avez probablement appris par la presse et p

# 1. Carga del Dataset

In [ ]:
! pip uninstall tensorflow

Found existing installation: tensorflow 2.11.0
Uninstalling tensorflow-2.11.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.8/dist-packages/tensorflow-2.11.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/tensorflow/*
Proceed (Y/n)? 

In [ ]:
! pip install tensorflow-gpu==2.4.0

In [ ]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [ ]:
YYYimport tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

SyntaxError: ignored

In [ ]:
import collections
import os

import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

In [ ]:
import os

# Load English data
english_sentences = load_data('europarl-v7.fr-en.en')
# Load French data
french_sentences = load_data('europarl-v7.fr-en.fr')

print('Dataset Loaded')
print(len(english_sentences))

dataset_size = int(len(english_sentences) / 6)

# Let's use a small dataset
english_sentences = english_sentences[:dataset_size]
french_sentences = french_sentences[:dataset_size]

In [ ]:
for sample_i in range(2):
    print('europarl-v7.fr-en.en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('europarl-v7.fr-en.fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

In [ ]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

# 2. Preprocesado

In [ ]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    t = Tokenizer()
    # fit the tokenizer on the documents
    t.fit_on_texts(x)
    return t.texts_to_sequences(x), t


# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

In [ ]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

In [ ]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

# 3. Entrenamiento

Primero usaremos una implementación de RNN many to many

In [ ]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1).astype('float32')])

print('`logits_to_text` function loaded.')

In [ ]:
import tensorflow as tf

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

learning_rate = 1e-3

input_seq = tf.keras.layers.Input(tmp_x.shape[1:])
rnn = tf.compat.v1.keras.layers.CuDNNLSTM(100, return_sequences=True)(input_seq)
# french_vocab_size
logits = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(french_vocab_size))(rnn)

model = tf.keras.models.Model(input_seq, tf.keras.layers.Activation('softmax')(logits))
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['accuracy'])

model.fit(tmp_x, preproc_french_sentences, batch_size=64, epochs=1, validation_split=0.2)

# Print prediction(s)

print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))

In [ ]:
english_sentences[0]

In [ ]:
y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
y_id_to_word[0] = '<PAD>'

sentence = 'he saw a old yellow truck'
sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=tmp_x.shape[-1], padding='post')
sentences = np.array([sentence[0], tmp_x[0]])
predictions = model.predict(sentences, len(sentences))
print(logits_to_text(predictions, french_tokenizer))
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))

In [ ]:
print(logits_to_text(model.predict(tmp_x[:])[0], french_tokenizer))

Añadimos una capa de Embedding previa a la capa de GRU:

In [ ]:
learning_rate = 1e-3

input_seq = tf.keras.layers.Input(tmp_x.shape[2:])
embedding = tf.keras.layers.Embedding(input_dim=english_vocab_size, output_dim=256)(input_seq)
rnn = tf.keras.layers.GRU(100, return_sequences=True)(embedding)
# french_vocab_size
logits = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(french_vocab_size))(rnn)

model = tf.keras.models.Model(input_seq, tf.keras.layers.Activation('softmax')(logits))
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['accuracy'])

model.fit(tmp_x, preproc_french_sentences, batch_size=64, epochs=1, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))

Ahora usamos LSTM Bidireccionales:

In [ ]:
learning_rate = 1e-3

input_seq = Input(tmp_x.shape[1:])
rnn = Bidirectional(GRU(100, return_sequences=True))(input_seq)
logits = TimeDistributed(Dense(french_vocab_size))(rnn)

model = Model(input_seq, Activation('softmax')(logits))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])

model.fit(tmp_x, preproc_french_sentences, batch_size=64, epochs=1, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))

Ahora vamos a implementar un traductor basado en seq2seq con Encoders y Decoders:

In [ ]:
from tensorflow.keras.layers import LSTM

learning_rate = 1e-3
    
#Encoder
encoder_input_seq = Input(shape=tmp_x.shape[1:])
encoder_output, state_h, state_c = LSTM(units=400, 
                              return_sequences=False,
                              return_state=True)(encoder_input_seq)
    
#Decoder  
decoder_input_seq = RepeatVector(max_french_sequence_length)(encoder_output)
decoder_out = LSTM(units=400,
                  return_sequences=True,
                  return_state=False)(decoder_input_seq, initial_state=[state_h, state_c])
logits = TimeDistributed(Dense(units=french_vocab_size))(decoder_out) 

#Model
model = Model(encoder_input_seq, Activation('softmax')(logits))
model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(lr=learning_rate),
              metrics=['accuracy'])

model.fit(tmp_x, preproc_french_sentences, batch_size=32, epochs=1, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(model.predict(tmp_x[:1])[0], french_tokenizer))